In [93]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install mistral_inference


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [115]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [116]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.chunk_size = 500
Settings.chunk_overlap = 25

In [117]:
documents = SimpleDirectoryReader("documents").load_data()


Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 137 0 (offset 0)
Ignoring wrong pointing object 177 0 (offset 0)
Ignoring wrong pointing object 179 0 (offset 0)
Ignoring wrong pointing object 181 0 (offset 0)
Ignoring wrong pointing object 202 0 (offset 0)
Ignoring wrong pointing object 287 0 (offset 0)
Ignoring wrong pointing object 289 0 (offset 0)
Ignoring wrong pointing object 311 0 (offset 0)
Ignoring wrong pointing object 313 0 (offset 0)
Ignoring wrong pointing object 315 0 (offset 0)
Ignoring wrong pointing object 391 0 (offset 0)
Ignoring wrong pointing object 412 0 (offset 0)


In [118]:
index = VectorStoreIndex.from_documents(documents)

In [119]:
top_k = 3
retriever = VectorIndexRetriever(index=index, similarity_top_k=top_k)

In [120]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)


In [123]:
query = "what is cancer"
response = query_engine.query(query)



In [124]:
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"
print(context)

Context:
Contents:
What is cancer?
Cancer  and its molecular  basis
What ar e the common cancers of human body?
What ar e the common causes of cancers?
Screening for  cancer
How is cancer  treated?
Prevention of cancer
Self-assessmentCANCER
You must have often wondered, what is cancer?
How is it caused? What can we do to prevent it?
How is it diagnosed and treated? Well, this chapter is 
there to address these issues only. Having awareness 
about the cancer is the first step one can take in 
preventing it.
What is cancer?
Cancers are a group of diseases associated with 
abnormal growth of cells.W ithout any check,
the disease may keep on progressing ultimately 
leading to pre-mature death. They can arise 
anywhere in the body and can affect people from
all age groups, socio-economic strata and race. 
Cancer is the leading cause of morbidity and 
mortality in the world. According to data by 
International Agency for Research on Cancer , there 
were 141 lakh new cancer cases, 82 lakh can

In [125]:

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [129]:
comment = "What is cancer"

context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"



# Format the prompt


prompt_template_w_context = lambda context, comment: f"""Please remove any text like "The picture can't be displayed.", file paths or any other text like "Source:ID card
Treatment: 1/1.

{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
[/INST]
"""
prompt = prompt_template_w_context(context, comment)
# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate text using the model
generated_ids = model.generate(
    inputs['input_ids'],  # Use the input_ids directly
    max_length=1500,
    num_return_sequences=1,
    temperature=0.7
)

# Decode the generated text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: index out of range in self

In [135]:

import os

from openai import OpenAI

client = OpenAI(
  api_key="your api key",  # this is also the default, it can be omitted
)



# Generate a response
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=150,
    temperature=0.7,
    n=1
)

# Extract and print the generated text
generated_text = response.choices[0].message["content"].strip()
print(generated_text)

# Additional information (usage details and full response)
print(response.model_dump_json(indent=2))


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}